In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.io as pio
from glob import glob # this will help us download the data in order to visualize it
#These functions will be vital in making the CNN
import tensorflow as tf 
from tensorflow import keras
from keras import models
from keras import layers
#For some image processing
import keras.preprocessing.image as kpi
#This will be important for processing the metadata
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [ ]:
#Starting with down loading the meta data
data_location = '/fs/ess/PAS2038/PHYSICS5680_OSU/student_data/armitage'
Meta = pd.DataFrame()
Meta_name = data_location + '/train.csv'
Meta = pd.read_csv(Meta_name,header=0)

In [ ]:
Meta['Random'] = np.random.randint(0,1000,size = len(Meta))

In [ ]:
Meta_data = Meta.iloc[:,1:13]
#Add this for reference
Meta_data['Random'] = Meta['Random']
Meta_data['Pawpularity'] = Meta['Pawpularity']
Meta = Meta_data
Meta

In [ ]:
from sklearn.model_selection import train_test_split
#Splitting into X(Features) Y(Pawpularity)
X_Meta = Meta.iloc[:,1:13].to_numpy()
Y_Meta = Meta['Pawpularity']


X_train,X_test,Y_train,Y_test = train_test_split(X_Meta,Y_Meta.values, test_size=0.2, shuffle = True)

In [ ]:
X = Meta.iloc[:,1:13]
X

In [ ]:
Y_test = np.array(Y_test)
Y_train = np.array(Y_train)

In [ ]:
X_train

In [ ]:
from sklearn import metrics
from sklearn.metrics import auc

def calc_performance_multi(true_labels,predicted_labels):
#
# Get the total number of unique labels
    num_labels = len(set(true_labels))
    confusionMatrix = nested_defaultdict(int,num_labels)
#
# Initialize all columns and rows
    for true_class in set(true_labels):
        for pred_class in set(true_labels):
            confusionMatrix[true_class][pred_class] = 0
# 
# Now calculate the confusion matrix
    for i in range(len(predicted_labels)):
        true_class = true_labels[i]      # this is either 0 or 1
        pred_class = predicted_labels[i]
        confusionMatrix[true_class][pred_class] += 1
#
# Get the recall, precision, ands F1 for each individual label
# - return both the "string report" (which you can print)
# - and the "dictionary report" (which you can use for averages and so on)
    report = classification_report(true_labels,predicted_labels)
    report_dict = classification_report(true_labels,predicted_labels,output_dict=True)
#
    results = {"confusionMatrix":confusionMatrix,"report":report,"report_dict":report_dict}
    return results


In [ ]:
def run_fitter_multi(estimator,X_train,y_train,X_test,y_test):
#
# Now fit to our training set
    estimator.fit(X_train,y_train)
#
# Now predict the classes and get the score for our traing set
    y_train_pred = estimator.predict(X_train)

#
# Now predict the classes and get the score for our test set
    y_test_pred = estimator.predict(X_test)
 
    results_test = calc_performance_multi(y_test,y_test_pred)
    results_train = calc_performance_multi(y_train,y_train_pred)
#
    return results_train,results_test

# Doing the random forest regressor here, but running to check max depth

In [ ]:
from sklearn.ensemble import RandomForestRegressor

dfError = []

#Here is the random forest regressor which aims to train on the RMSE, from the Meta Data
for depth in range (5,20):
    estimator = RandomForestRegressor(n_estimators=100,random_state=42,max_depth= depth)
    estimator.fit(X_train,Y_train)
    
    y_train_pred = estimator.predict(X_train)
    y_test_pred = estimator.predict(X_test)
    
    train_RMSE = np.sqrt(mean_squared_error(Y_train,y_train_pred))
    test_RMSE = np.sqrt(mean_squared_error(Y_test,y_test_pred))
    dfError.append({'max_depth':depth,
                             'train_RMSE':train_RMSE,
                             'test_RMSE':test_RMSE
                             })

In [ ]:
df = pd.DataFrame(dfError)

In [ ]:
df

In [ ]:
fig = px.line(df, x='max_depth', y=['train_RMSE','test_RMSE'], title='RMSE vs Iteration')
fig.show()
#

In [ ]:
X_train

In [ ]:
dffeatures= pd.DataFrame()
dffeatures['Features'] = ['Eyes','Face','Near','Action','Accessory','Group','Collage','Human','Occlusion','Info','Blur','Random']

In [ ]:
dffeatures["Importance"] = estimator.feature_importances_

In [ ]:
dffeatures

import matplotlib.pyplot as plt

#plt.plot(X_train,Y_train)

In [ ]:
fig = px.histogram(dffeatures,x='Features', y="Importance",
                      title='Feature Importance')
fig.show()



In [ ]:
Extracted_Features = pd.read_csv('Copy of Pet_Features.csv',header = None,delimiter = ',')

In [ ]:
Extracted_Features

In [ ]:
Meta_data['Pawpularity']

In [ ]:
Extracted_Features['Pawpularity'] = Meta_data['Pawpularity']

In [ ]:
Extracted_Features